# Text Mining Workshop Notebook
This will be the notebook that we use for the text mining lesson. The past lesson is the other Jupyter notebook in this repository, `ucsd_newspapers.ipynb`

In [2]:
import os
import numpy as np
import pandas as pd
import nltk
import nltk.data

In [3]:
# We need to point to nltk pre-trained data we have in this repo
sent_detector = nltk.data.load('nltk_data/english.pickle')


In [10]:
sentences = text = '''
    Punkt knows that the periods in Mr. Smith and Johann S. Bach do not mark sentence boundaries.  
    And sometimes sentences can start with non-capitalized words.  i is a good variable name. "And now for something completely different" 
    '''

In [11]:
print('\n\n'.join(sent_detector.tokenize(text.strip())))

Punkt knows that the periods in Mr. Smith and Johann S. Bach do not mark sentence boundaries.

And sometimes sentences can start with non-capitalized words.

i is a good variable name.

"And now for something completely different"
